In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import scipy as s
from netCDF4 import Dataset
import os
from vis_io import get_vtu, write_vtu, get_vts, write_vts, get_mesh
from vec_io import read_grid, write_grid, read_vec, write_vec
import vtk
from data.data import SphericalDataset
from vtkmodules.util import numpy_support
import scipy.optimize as op

In [2]:
def deg2rad(deg):
  return deg * np.pi / 180

# for numerical isse
def mysin(a):
  val = np.sin(a)
  val = close_round(val, 0)
  val = close_round(val, 1)
  return val

def mycos(a):
  val = np.cos(a)
  val = close_round(val, 0)
  val = close_round(val, 1)
  return val

def close_round(val, test_val, abs_bounds=1e-12):
  isclose = np.abs(test_val - val) < abs_bounds
  # print(isclose)
  if isinstance(val, float) or isinstance(val, int) or np.float32:
    val_cp = test_val if isclose else val
  else:
    val_cp = val.copy()
    val_cp[isclose] = test_val
  return val_cp

def sph2car(r, theta, phi):
  # x_coef = np.sin(phi)*np.cos(theta)
  # y_coef = np.around(np.sin(phi)*np.sin(theta), decimals=10)
  # z_coef = np.around(np.cos(phi), decimals=10)
  x = r*mysin(phi)*mycos(theta)
  y = r*mysin(phi)*mysin(theta)
  z = r*mycos(phi)
  
  return np.array([x,y,z])

def car2sph(x, y, z):
  # assert (x or y) != 0
  r = np.sqrt(x*x + y*y + z*z)
  theta = np.arctan(y/x)
  phi = np.arctan(np.sqrt(x*x + y*y) / z)
  # if x > 0:
  #   phi = np.arctan(y/x)
  # elif x < 0 and y >= 0:
  #   phi = np.arctan(y/x)+np.pi
  # elif x < 0 and y < 0:
  #   phi == np.arctan(y/x)-np.pi
  # elif x == 0 and y > 0:
  #   phi = np.pi/2
  # elif x == 0 and y < 0:
  #   phi = -np.pi/2

  return np.array([r,theta,phi])
# //
# //		    6________7  high-vtx
# //		   /|       /|
# //		  / |      / |
# //		4/_______5/  |
# //		|  2|___ |___|3
# //		|  /     |  /
# //		| /      | /
# //		|/_______|/
# //		0        1
# //  low_vtx

# //
# //		 011_________111  high-vtx
# //		   /|       /|
# //		  / |      / |
# //	001/_____101/  |
# //		|010|___ |___|110
# //		|  /     |  /
# //		| /      | /
# //		|/_______|/
# //	000       100


def trilerp(coord: np.ndarray, low_vtx: np.ndarray, high_vtx: np.ndarray, cell_val: np.ndarray):
  # x0,y0,z0 = low_vtx
  # x1,y1,z1 = high_vtx
  # x, y, z = coord

  # low_vtx, high_vtx, coord = normalization(np.array([low_vtx, high_vtx, coord]), new_min=0, dim=0)

  # TODO: Why trilerp doesn't work if low and high are not normalized to 1
  x, y, z = (coord - low_vtx) / (high_vtx - low_vtx)
  low_vtx = np.zeros(3)
  high_vtx = np.ones(3)
  x0,y0,z0 = low_vtx
  x1,y1,z1 = high_vtx

  v000 = cell_val[0,0,0]
  v100 = cell_val[1,0,0]
  v010 = cell_val[0,1,0]
  v110 = cell_val[1,1,0]
  v001 = cell_val[0,0,1]
  v101 = cell_val[1,0,1]
  v011 = cell_val[0,1,1]
  v111 = cell_val[1,1,1]
  # print(x, y, z)
  # print(x0, y0, z0)
  # print(x1, y1, z1)
  # print((x0-x1)*(y0-y1)*(z0-z1))
  # print(((x0-x1)*(y0-y1)*(z0-z1)))
  # for cord in [v000, v100, v010, v110, v001, v101, v011, v111]:
  #   print(cord)
  a0 = (
    (-v000*x1*y1*z1 + v001*x1*y1*z0 + v010*x1*y0*z1 - v011*x1*y0*z0 + 
    v100*x0*y1*z1 - v101*x0*y1*z0 - v110*x0*y0*z1 + v111*x0*y0*z0)
  )
  a1 = (
    (v000*z1*y1 - v001*z0*y1 - v010*z1*y0 + v011*z0*y0 - 
    v100*z1*y1 + v101*z0*y1 + v110*z1*y0 - v111*z0*y0)
  )
  a2 = (
    (v000*x1*z1 - v001*x1*z0 - v010*x1*z1 + v011*x1*z0 - 
    v100*x0*z1 + v101*x0*z0 + v110*x0*z1 - v111*x0*z0)
  )
  a3 = (
    (v000*x1*y1 - v001*x1*y1 - v010*x1*y0 + v011*x1*y0 - 
    v100*x0*y1 + v101*x0*y1 + v110*x0*y0 - v111*x0*y0)
  )
  a4 = (
    (-v000*z1 + v001*z0 + v010*z1 - v011*z0 + 
    v100*z1 - v101*z0 - v110*z1 + v111*z0)
  )
  a5 = (
    (-v000*y1 + v001*y1 + v010*y0 - v011*y0 + 
    v100*y1 - v101*y1 - v110*y0 + v111*y0)
  )
  a6 = (
    (-v000*x1 + v001*x1 + v010*x1 - v011*x1 + 
    v100*x0 - v101*x0 - v110*x0 + v111*x0)
  )
  a7 = (
    (v000 - v001 - v010 + v011 -v100 + v101 + v110 - v111)
  )
  interpolant = (a0 + a1*x + a2*y + a3*z + a4*x*y + a5*x*z + a6*y*z + a7*x*y*z) / ((x0-x1)*(y0-y1)*(z0-z1))
  coeff = np.array([a0, a1, a2, a3, a4, a5, a6, a7]).T
  return interpolant, coeff

# G
def comp2phys(comp: np.ndarray, vtx_low, vtx_high, cell_coords):
  phys_est_x, a = trilerp(comp, vtx_low, vtx_high, cell_coords[..., 0])
  phys_est_y, b = trilerp(comp, vtx_low, vtx_high, cell_coords[..., 1])
  phys_est_z, c = trilerp(comp, vtx_low, vtx_high, cell_coords[..., 2])
  return np.array([phys_est_x, phys_est_y, phys_est_z]), np.array([a,b,c])

# F
def diff_phys(comp: np.ndarray, phys: np.ndarray, vtx_low, vtx_high, cell_coords):
  phys_est, coeff = comp2phys(comp, vtx_low, vtx_high, cell_coords)
  return phys - phys_est

def diff_phys_dim(comp: np.ndarray, phys: np.ndarray, vtx_low, vtx_high, cell_coords_dim):
  phys_est = trilerp(comp, vtx_low, vtx_high, cell_coords_dim)
  return phys - phys_est
# op.root_scalar(diff_phys, method='newton')

def diff_phys_dim_inv_jac():
  pass

class CurviInterpField:
  def __init__(self):
    pass

class CurviInterpCell:
  def __init__(self, min_comp, max_comp, phys_coords):
    self.min_comp = np.array(min_comp)
    self.max_comp = np.array(max_comp)
    self.phys = phys_coords
    self.init_comp = min_comp + (max_comp - min_comp) / 2
    self.coeff = None

    # xs = [min_comp[0], max_comp[0]]
    # ys = [min_comp[1], max_comp[1]]
    # zs = [min_comp[2], max_comp[2]]
    # self.interp = RegularGridInterpolator((xs, ys, zs), phys_coords)

  # Newton's method to find computational coordinate given physical cooridnate
  def phys2comp(self, phys: np.ndarray, init_comp=None, tol=1.48e-8, maxiter=50, rtol=0.0):
    if init_comp is None:
      init_comp = self.init_comp

    comp = init_comp
    goal_diff = np.zeros(3)
    for i in range(maxiter):
      phys_est, coeff = self.comp2phys(comp)
      diff_funcval = (phys_est - phys)

      jac_inv = self.get_jacobian_inv(comp, coeff)

      new_comp = comp - (jac_inv @ diff_funcval.reshape(3, 1)).flatten()
      # print(phys_est.shape, coeff.shape, phys.shape)
      # print(new_comp.shape, comp.shape)
      # print(jac_inv.shape, diff_funcval.shape)
      # print(f'phys_est: {phys_est} comp: {new_comp}  error: {np.abs(diff_funcval-goal_diff)}')
      # if np.all(np.isclose(new_comp, comp, rtol=rtol, atol=tol)):
      if np.all(np.isclose(diff_funcval, goal_diff, rtol=rtol, atol=tol)):
        print(f'iter {i}')
        return new_comp
      comp = new_comp
    return comp

  # def phys2comp(self, phys):
  #   return self.find_comp(phys)

  def comp2phys(self, comp):
    phys_est, coeff = trilerp(comp, self.min_comp, self.max_comp, self.phys)
    # phys_est_y, b = trilerp(comp, self.min_comp, self.max_comp, self.phys[..., 1])
    # phys_est_z, c = trilerp(comp, self.min_comp, self.max_comp, self.phys[..., 2])
    # self.coeff = np.array([a,b,c])
    # return np.array([phys_est_x, phys_est_y, phys_est_z]), np.array([a,b,c])
    # print(phys_est.shape, coeff.shape)
    # self.interp(comp)
    return phys_est, coeff
  
  def get_jacobian(self, comp, coeff):
    a,b,c = coeff
    return np.array([
      [
        a[1] + a[4]*comp[1] + a[5]*comp[2] + a[7]*comp[1]*comp[2],
        a[2] + a[4]*comp[0] + a[6]*comp[2] + a[7]*comp[0]*comp[2],
        a[3] + a[5]*comp[0] + a[6]*comp[1] + a[7]*comp[0]*comp[1],
      ],
      [
        b[1] + b[4]*comp[1] + b[5]*comp[2] + b[7]*comp[1]*comp[2],
        b[2] + b[4]*comp[0] + b[6]*comp[2] + b[7]*comp[0]*comp[2],
        b[3] + b[5]*comp[0] + b[6]*comp[1] + b[7]*comp[0]*comp[1],
      ],
      [
        c[1] + c[4]*comp[1] + c[5]*comp[2] + c[7]*comp[1]*comp[2],
        c[2] + c[4]*comp[0] + c[6]*comp[2] + c[7]*comp[0]*comp[2],
        c[3] + c[5]*comp[0] + c[6]*comp[1] + c[7]*comp[0]*comp[1],
      ]
    ])
  
  def get_jacobian_inv(self, comp, coeff):
    jac = self.get_jacobian(comp, coeff)
    # np_det = np.linalg.det(jac)
    # np_inv = np.linalg.inv(jac)
    det = (-jac[0,0]*jac[1,1]*jac[2,2] - jac[0,1]*jac[1,2]*jac[2,0] - jac[0,2]*jac[1,0]*jac[2,1] + 
            jac[0,2]*jac[1,1]*jac[2,0] + jac[0,1]*jac[1,0]*jac[2,2] + jac[0,0]*jac[1,2]*jac[2,1])
    inv = np.array([
      [
        jac[1,1]*jac[2,2] - jac[1,2]*jac[2,1],
        -jac[0,1]*jac[2,2] + jac[0,2]*jac[2,1],
        jac[0,1]*jac[1,2] - jac[0,2]*jac[1,1],
      ],
      [
        -jac[1,0]*jac[2,2] + jac[1,2]*jac[2,0],
        jac[0,0]*jac[2,2] - jac[0,2]*jac[2,0],
        -jac[0,0]*jac[1,2] + jac[0,2]*jac[1,0],
      ],
      [
        jac[1,0]*jac[2,1] - jac[1,1]*jac[2,0],
        jac[0,0]*jac[2,1] - jac[0,1]*jac[2,0],
        jac[0,0]*jac[1,1] - jac[0,1]*jac[1,0],
      ],
    ])
    # print(jac)
    # print(np_inv)
    # print(inv/det)
    # print(np_det)
    # print(det)
    return inv/det

In [5]:
mt = Dataset("/fs/project/PAS0027/mantle/data/spherical010.nc", format="netcdf4")
# r:
# lon: [0, 360] theta (math)
# lat: [-90, 90] phi (math)
r = mt.variables['r'][:]
lon = mt.variables['lon'][:]
lat = mt.variables['lat'][:] + 90
lat_acsend = np.flip(lat)
temp = mt.variables['temperature'][:]

print('lon:')
isSorted(lon)
print('lat:')
isSorted(lat)
print('r:')
isSorted(r)

mt.close()

lon:
ascending sorted
lat:
decending sorted
r:
ascending sorted


In [13]:
grid_size = np.array([len(r), len(lon), len(lat_acsend)], dtype=np.int32)
r.astype(np.float32)

masked_array(data=[3485.  , 3492.25, 3506.75, 3521.26, 3535.76, 3550.26,
                   3564.76, 3579.27, 3593.77, 3608.27, 3622.77, 3637.28,
                   3651.78, 3666.28, 3680.78, 3695.29, 3709.79, 3724.29,
                   3738.79, 3753.3 , 3767.8 , 3782.3 , 3796.8 , 3811.31,
                   3825.81, 3840.31, 3854.81, 3869.32, 3883.82, 3898.32,
                   3912.82, 3927.33, 3941.83, 3956.33, 3970.83, 3985.34,
                   3999.84, 4014.34, 4028.84, 4043.35, 4057.85, 4072.35,
                   4086.85, 4101.36, 4115.86, 4130.36, 4144.86, 4159.37,
                   4173.87, 4188.37, 4202.87, 4217.38, 4231.88, 4246.38,
                   4260.88, 4275.39, 4289.89, 4304.39, 4318.89, 4333.4 ,
                   4347.9 , 4362.4 , 4376.9 , 4391.41, 4405.91, 4420.41,
                   4434.91, 4449.42, 4463.92, 4478.42, 4492.92, 4507.43,
                   4521.93, 4536.43, 4550.93, 4565.44, 4579.94, 4594.44,
                   4608.94, 4623.45, 4637.95, 4652.

In [39]:
# downsample a bit
r = r[:]
lon = lon[:]
# remove poles to avoid same physcial coordinates on the poles
lat = lat[range(10, len(lat)-10)]
lat_acsend = lat_acsend[range(10, len(lat_acsend)-10)]

In [107]:
# output comp space
dims = [len(r), len(lon), len(lat_acsend)]
coords = get_mesh(r, lon, lat_acsend).reshape(-1, 3)
temp_val = temp.flatten()[:len(coords)]

# output cart space
lon_rad = np.radians(lon)
lat_rad = np.radians(lat_acsend)
coords_rad = get_mesh(r, lon_rad, lat_rad).reshape(-1, 3)

cart_coords = sph2car(coords_rad[:,0], coords_rad[:,1], coords_rad[:,2])
for i in range(len(cart_coords)):
  cart_coords[i] = cart_coords[i][..., None]
cart_coords = np.concatenate(cart_coords, axis=-1)

meshe generated: (201, 360, 140, 3)
meshe generated: (201, 360, 140, 3)


In [ ]:
curv = get_vts(dims, coords, {})
write_vts("test_comp.vts", curv)
cart = get_vts(dims, cart_coords, {})
write_vts("test_cart.vts", cart)

In [120]:
write_vec('../test/src/data/mantle_phys.vec', *dims, vecs=cart_coords)
write_grid('../test/src/data/mantle_comp.grid', r, lon, lat_acsend)
xyz = read_vec('../test/src/data/mantle_phys.vec')
xg, yg, zg = read_grid('../test/src/data/mantle_comp.grid')

In [5]:
xyz = read_vec('../test/src/data/mantle_phys.vec')
xg, yg, zg = read_grid('../test/src/data/mantle_comp.grid')
xyz_comp = get_mesh(xg, yg, zg)

meshe generated: (201, 360, 140, 3)


In [16]:
s = np.array(xyz.shape, dtype=np.int32)

In [27]:
s.prod()

30391200

In [6]:
xyz[20,12,88]

array([ 3490.3113,   744.0171, -1208.5039], dtype=float32)

In [7]:
xyz.reshape(-1, 3)[20*50400 + 140*12 + 88]

array([ 3490.3113,   744.0171, -1208.5039], dtype=float32)

In [9]:
20*50400 + 140*12 + 88

1009768

In [8]:
xyz_comp[51,23,85]

array([4217.38    ,   23.064066,  105.674   ], dtype=float32)

In [129]:
display(xg[:10], r[:10])
display(xyz.reshape(-1, 3)[:10], cart_coords[:10])

array([3485.  , 3492.25, 3506.75, 3521.26, 3535.76, 3550.26, 3564.76,
       3579.27, 3593.77, 3608.27], dtype=float32)

masked_array(data=[3485.  , 3492.25, 3506.75, 3521.26, 3535.76, 3550.26,
                   3564.76, 3579.27, 3593.77, 3608.27],
             mask=False,
       fill_value=1e+20)

array([[1175.865 ,    0.    , 3280.635 ],
       [1233.5667,    0.    , 3259.377 ],
       [1290.9407,    0.    , 3237.0815],
       [1347.856 ,    0.    , 3213.7998],
       [1404.3516,    0.    , 3189.5173],
       [1460.41  ,    0.    , 3164.242 ],
       [1516.0685,    0.    , 3137.955 ],
       [1571.1996,    0.    , 3110.7163],
       [1625.8417,    0.    , 3082.5093],
       [1679.9774,    0.    , 3053.3425]], dtype=float32)

masked_array(
  data=[[1175.86494003,    0.        , 3280.63509748],
        [1233.56660539,    0.        , 3259.3770003 ],
        [1290.94070045,    0.        , 3237.08157264],
        [1347.85597732,    0.        , 3213.7998171 ],
        [1404.35160221,    0.        , 3189.51745212],
        [1460.40998534,    0.        , 3164.24203795],
        [1516.06844094,    0.        , 3137.95498412],
        [1571.19964577,    0.        , 3110.71642441],
        [1625.84166108,    0.        , 3082.50935004],
        [1679.97747418,    0.        , 3053.34254322]],
  mask=False,
  fill_value=1e+20)